#Importing Libraries

In [1]:
import tensorflow as tf         #library for building cnn
import matplotlib.pyplot as plt #for visualization
import pandas as pd             #for working with datasets
import seaborn as sns           #for data visualization

#Data Pre-processing
#Training Image pre-processing

In [2]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 70296 files belonging to 38 classes.


#Validation Image Pre-processing

In [3]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 17572 files belonging to 38 classes.


In [4]:
training_set

<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>

In [5]:
for x,y in training_set:
    print(x, x.shape)
    print(y, y.shape)
    break

tf.Tensor(
[[[[212.5  211.5  207.5 ]
   [210.25 209.25 205.25]
   [206.75 205.75 201.75]
   ...
   [138.5  133.5  129.5 ]
   [109.   104.   100.  ]
   [105.   100.    96.  ]]

  [[213.25 212.25 208.25]
   [212.5  211.5  207.5 ]
   [215.   214.   210.  ]
   ...
   [131.75 126.75 122.75]
   [124.5  119.5  115.5 ]
   [134.5  129.5  125.5 ]]

  [[220.5  219.5  215.5 ]
   [211.   210.   206.  ]
   [211.25 210.25 206.25]
   ...
   [127.5  122.5  118.5 ]
   [123.   118.   114.  ]
   [131.25 126.25 122.25]]

  ...

  [[186.25 183.25 178.25]
   [185.5  182.5  177.5 ]
   [181.   178.   173.  ]
   ...
   [113.75 108.75 105.75]
   [117.25 112.25 109.25]
   [138.5  133.5  130.5 ]]

  [[182.75 179.75 174.75]
   [184.5  181.5  176.5 ]
   [186.5  183.5  178.5 ]
   ...
   [116.75 111.75 108.75]
   [115.   110.   107.  ]
   [126.   121.   118.  ]]

  [[184.5  181.5  176.5 ]
   [186.5  183.5  178.5 ]
   [184.75 181.75 176.75]
   ...
   [115.75 110.75 107.75]
   [113.5  108.5  105.5 ]
   [115.   110.   10

### TO avoid Overshooting
#1. Choose small learning rate default 0.001 we are taking 0.0001
#2. There may be chance of Underfitting, so increase number of neuron
#3. Add more Convolution layer to extract more features from images there may be possibly that model unable to capture relevant feature or model is confusing due to lack of feature so feed with more feature.

#Building Model

In [8]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.models import Sequential

In [9]:
model = Sequential()

#Building Convolution layer

In [11]:
model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
model.add(Conv2D(filters=32,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [12]:
model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=64,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [13]:
model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [14]:
model.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=256,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [15]:
model.add(Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=512,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

#To avoid Overfitting "It's often used in larger networks where the risk of overfitting is higher due to the large number of parameters.

In [16]:
model.add(Dropout(0.25))

#Converting it to a 1D array to add it in the hidden layer activation

In [17]:
model.add(Flatten())

#hidden layer activation

In [18]:
model.add(Dense(units=1500,activation='relu')) 

In [19]:
model.add(Dropout(0.4))

#Output layer

In [20]:
model.add(Dense(units=38,activation='softmax'))

#Compiling Model

In [22]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                    

#Model Training

In [28]:
training_history = model.fit(x=training_set,validation_data=validation_set,epochs=10)

Epoch 1/10
2197/2197 [==============================] - 2189s 997ms/step - loss: 3.6373 - accuracy: 0.0277 - val_loss: 3.6362 - val_accuracy: 0.0286
Epoch 2/10
2197/2197 [==============================] - 2205s 1s/step - loss: 3.6366 - accuracy: 0.0279 - val_loss: 3.6361 - val_accuracy: 0.0287
Epoch 3/10
1851/2197 [========================>.....] - ETA: 4:57 - loss: 3.6363 - accuracy: 0.0276

KeyboardInterrupt: 

#Model Evaluation
##Model evaluation on Training set

In [ ]:
train_loss,train_acc = model.evaluate(training_set)

In [ ]:
print(train_loss,train_acc)

#Model on validation set

In [ ]:
val_loss,val_acc = model.evaluate(validation_set)

In [ ]:
print(val_loss,val_acc)

#Saving Model

In [ ]:
model.save("leaf_disease_trained_model.keras")
model.save("leaf_disease_trained_model.h5")

In [ ]:
training_history.history

#Recording History in json

In [ ]:
import json
with open("training_hist.json","w") as f:
    json.dump(training_history.history,f)

#Accuracy visualization

In [ ]:
import matplotlib.pyplot as plt
epochs = [i for i in range(1,11)]
plt.plot(epochs,training_history.history['accuracy'],color='red',label='Training Accuracy')
plt.plot(epochs,training_history.history['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel("No. of Epochs")
plt.ylabel("Accuracy Result")
plt.title("Visualization of Accuracy Result")
plt.legend()
plt.show()

#Some other metrics for model evaluation

In [ ]:
class_name = validation_set.class_names
# class_name